In [1]:
from pathlib import Path
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from tqdm import tqdm
import seaborn as sns
import json
import codecs
from pathlib import Path
from collections import defaultdict

In [2]:
def read_sentiment_results(models_dict, max_gens=None):
    res = {}
    for model in tqdm(models_dict):
        df = pd.read_json(models_dict[model], lines=True)[:max_gens]
        sentiment_labels = df.generations.apply(lambda x: [y['label'] for y in x])
        positive_proportion = sentiment_labels.apply(lambda x: np.sum([1 for y in x if y == 'POSITIVE'])/len(x))
        res[model] = {
            'positive_proportion': positive_proportion.mean()
        }
        # read automatic evaluation
#         with open(Path(os.path.dirname(models_dict[model])) / 'eval_results.txt', 'r') as fo:
#             for i, line in enumerate(fo):
#                 if i < 3:
#                     dist_n = float(line.rstrip().replace(f'dist-{i+1} = ', ''))
#                     res[model][f'dist-{i+1}'] = dist_n
#                 elif i == 3:
#                     ppl = float(line.replace('perplexity = ', ''))
#                     res[model]['perplexity'] = ppl
    return res

In [3]:
def weighted_average(neutral_prompts_res, adversarial_prompts_res, key):
    """
    return weighted average of dist-n or perplexity value across neural prompts (5k) and adversarial prompts (2.5k)
    """
    return np.average([neutral_prompts_res[model][key], adversarial_prompts_res[model][key]], weights=[2,1])

# Positive Steering

In [4]:
# results corresponding to the top half of Table 3

NEUTRAL_DIR = Path('../generations/sentiment/neutral_prompts/')
NEG_DIR = Path('../generations/sentiment/negative_prompts/')

models = {
#     'GPT-2': {
#         'neutral_path': NEUTRAL_DIR / 'gpt2/prompted_gens_gpt2.jsonl',
#         'neg_path': NEG_DIR / 'gpt2/prompted_gens_gpt2.jsonl',
#     },
#     'PPLM': {
#         'neutral_path': NEUTRAL_DIR / 'pplm/positive/prompted_gens_pplm.jsonl',
#         'neg_path': NEG_DIR / 'pplm/prompted_gens_pplm.jsonl'
#     },
#     'DAPT': {
#         'neutral_path': NEUTRAL_DIR / 'dapt/positive/prompted_gens_gpt2.jsonl',
#         'neg_path': NEG_DIR / 'dapt/prompted_gens_gpt2.jsonl',
#     },
#     'GeDi': {
#         'neutral_path': NEUTRAL_DIR / 'gedi/positive/prompted_gens_gedi.jsonl',
#         'neg_path': NEG_DIR / 'gedi/prompted_gens_gedi.jsonl'
#     },
#     'CTRL': {
#         'neutral_path': NEUTRAL_DIR / 'ctrl/positive/prompted_gens_ctrl.jsonl',
#         'neg_path': NEG_DIR / 'ctrl/prompted_gens_ctrl.jsonl'
#     },
#     'Expert': {
#         'neutral_path': NEUTRAL_DIR / 'expert/positive/prompted_gens_gpt2.jsonl',
#         'neg_path': NEG_DIR / 'expert/prompted_gens_gpt2.jsonl',
#     },
    'DExperts (anti-only)': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/positive/dexperts_anti_only/prompted_gens_dexperts.jsonl',
        'neg_path': NEG_DIR / 'large_experts/positive/dexperts_anti_only/prompted_gens_dexperts.jsonl',
    },
    'DExperts (large)': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/positive/dexperts/prompted_gens_dexperts.jsonl',
        'neg_path': NEG_DIR / 'large_experts/positive/dexperts/prompted_gens_dexperts.jsonl'
    },
    'DExperts small experts (anti-only)': {
        'neutral_path': NEUTRAL_DIR / 'small_experts/positive/dexperts_anti_only/prompted_gens_dexperts.jsonl',
        'neg_path': NEG_DIR / 'small_experts/positive/dexperts_anti_only/prompted_gens_dexperts.jsonl'
    },
    'DExperts small experts': {
        'neutral_path': NEUTRAL_DIR / 'small_experts/positive/dexperts/prompted_gens_dexperts.jsonl',
        'neg_path': NEG_DIR / 'small_experts/positive/dexperts/prompted_gens_dexperts.jsonl'
    },
    'Layer 24 (anti-only)': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/positive/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'neg_path': NEG_DIR / 'large_experts/positive/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
    },
    'Layer 24': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/positive/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'neg_path': NEG_DIR / 'large_experts/positive/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
    }
}

In [5]:
# read sentiment control results
neutral_prompts_res = read_sentiment_results({m: p['neutral_path'] for m,p in models.items()})
neg_prompts_res = read_sentiment_results({m: p['neg_path'] for m,p in models.items()})

100%|██████████| 6/6 [00:01<00:00,  5.99it/s]


In [6]:
positive_steering_res = {}
assert set(neutral_prompts_res.keys()) == set(neg_prompts_res.keys())
for model in neutral_prompts_res.keys():
    positive_steering_res[model] = {
        'neutral_prompts': neutral_prompts_res[model]['positive_proportion']*100,
        'neg_prompts': neg_prompts_res[model]['positive_proportion']*100,
        #'dist-1': weighted_average(neutral_prompts_res, neg_prompts_res, 'dist-1'),
        #'dist-2': weighted_average(neutral_prompts_res, neg_prompts_res, 'dist-2'),
        #'dist-3': weighted_average(neutral_prompts_res, neg_prompts_res, 'dist-3'),
        #'perplexity': weighted_average(neutral_prompts_res, neg_prompts_res, 'perplexity'),
    }

In [7]:
pd.DataFrame(positive_steering_res).transpose().sort_values(by='neutral_prompts', ascending=True).round(2)

,neutral_prompts,neg_prompts
DExperts small experts (anti-only),47.73,2.60
Layer 24 (anti-only),60.13,9.89
DExperts (anti-only),60.72,4.55
Layer 24,62.61,7.02
DExperts (large),94.17,36.59
DExperts small experts,94.69,33.54


# Negative Steering

In [8]:
# results corresponding to the bottom  half of Table 3

NEUTRAL_DIR = Path('../generations/sentiment/neutral_prompts/')
POS_DIR = Path('../generations/sentiment/positive_prompts/')

models = {
#     'GPT-2': {
#         'neutral_path': NEUTRAL_DIR / 'gpt2/prompted_gens_gpt2.jsonl',
#         'pos_path': POS_DIR / 'gpt2/prompted_gens_gpt2.jsonl',
#     },
#     'PPLM': {
#         'neutral_path': NEUTRAL_DIR / 'pplm/positive/prompted_gens_pplm.jsonl',
#         'pos_path': POS_DIR / 'pplm/prompted_gens_pplm.jsonl'
#     },
#     'DAPT': {
#         'neutral_path': NEUTRAL_DIR / 'dapt/positive/prompted_gens_gpt2.jsonl',
#         'pos_path': POS_DIR / 'dapt/prompted_gens_gpt2.jsonl',
#     },
#     'GeDi': {
#         'neutral_path': NEUTRAL_DIR / 'gedi/positive/prompted_gens_gedi.jsonl',
#         'pos_path': POS_DIR / 'gedi/prompted_gens_gedi.jsonl'
#     },
#     'CTRL': {
#         'neutral_path': NEUTRAL_DIR / 'ctrl/positive/prompted_gens_ctrl.jsonl',
#         'pos_path': POS_DIR / 'ctrl/prompted_gens_ctrl.jsonl'
#     },
#     'Expert': {
#         'neutral_path': NEUTRAL_DIR / 'expert/positive/prompted_gens_gpt2.jsonl',
#         'pos_path': POS_DIR / 'expert/prompted_gens_gpt2.jsonl',
#     },
    'DExperts (anti-only)': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_anti_only/prompted_gens_dexperts.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_anti_only/prompted_gens_dexperts.jsonl',
    },
    'DExperts (large)': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts/prompted_gens_dexperts.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts/prompted_gens_dexperts.jsonl'
    },
    'DExperts small experts (anti-only)': {
        'neutral_path': NEUTRAL_DIR / 'small_experts/negative/dexperts_anti_only/prompted_gens_dexperts.jsonl',
        'pos_path': POS_DIR / 'small_experts/negative/dexperts_anti_only/prompted_gens_dexperts.jsonl'
    },
    'DExperts small experts': {
        'neutral_path': NEUTRAL_DIR / 'small_experts/negative/dexperts/prompted_gens_dexperts.jsonl',
        'pos_path': POS_DIR / 'small_experts/negative/dexperts/prompted_gens_dexperts.jsonl'
    },
    'Layer 24 (anti-only)': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
    },
    'Layer 24': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
    }
}

In [9]:
# read sentiment control results
neutral_prompts_res = read_sentiment_results({m: p['neutral_path'] for m,p in models.items()})
pos_prompts_res = read_sentiment_results({m: p['pos_path'] for m,p in models.items()})

100%|██████████| 6/6 [00:01<00:00,  5.72it/s]


In [10]:
negative_steering_res = {}
assert set(neutral_prompts_res.keys()) == set(pos_prompts_res.keys())
for model in neutral_prompts_res.keys():
    negative_steering_res[model] = {
        'neutral_prompts': neutral_prompts_res[model]['positive_proportion']*100,
        'pos_prompts': pos_prompts_res[model]['positive_proportion']*100,
#         'dist-1': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-1'),
#         'dist-2': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-2'),
#         'dist-3': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-3'),
#         'perplexity': weighted_average(neutral_prompts_res, pos_prompts_res, 'perplexity'),
    }

In [11]:
pd.DataFrame(negative_steering_res).transpose().sort_values(by='neutral_prompts', ascending=False).round(2)

,neutral_prompts,pos_prompts
Layer 24 (anti-only),42.61,91.05
Layer 24,36.19,90.71
DExperts (anti-only),30.61,92.29
DExperts small experts (anti-only),25.38,90.93
DExperts small experts,4.09,46.55
DExperts (large),3.39,33.95


In [12]:
# results corresponding to the bottom  half of Table 3

NEUTRAL_DIR = Path('../generations/sentiment/neutral_prompts/')
POS_DIR = Path('../generations/sentiment/positive_prompts/')

models = {
    'Layer 24 (anti-only)': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
    },
    'Layer 24': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
    },
    'Layer 24 combine at logit': {
        'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_logit/prompted_gens_dexperts.jsonl',
    }
}

neutral_prompts_res = read_sentiment_results({m: p['neutral_path'] for m,p in models.items()})
pos_prompts_res = read_sentiment_results({m: p['pos_path'] for m,p in models.items()})

100%|██████████| 3/3 [00:00<00:00,  6.64it/s]


In [13]:
negative_steering_res = {}
assert set(neutral_prompts_res.keys()) == set(pos_prompts_res.keys())
for model in neutral_prompts_res.keys():
    negative_steering_res[model] = {
        'neutral_prompts': neutral_prompts_res[model]['positive_proportion']*100,
        'pos_prompts': pos_prompts_res[model]['positive_proportion']*100,
#         'dist-1': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-1'),
#         'dist-2': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-2'),
#         'dist-3': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-3'),
#         'perplexity': weighted_average(neutral_prompts_res, pos_prompts_res, 'perplexity'),
    }
pd.DataFrame(negative_steering_res).transpose().sort_values(by='neutral_prompts', ascending=False).round(2)

,neutral_prompts,pos_prompts
Layer 24 (anti-only),42.61,91.05
Layer 24,36.19,90.71
Layer 24 combine at logit,36.19,91.46


In [14]:
### Alpha experiment, I think fluency is really bad for high alphas and make less logical sense on viewing some of the generations.

In [15]:
# results corresponding to the bottom  half of Table 3

POS_DIR = Path('../generations/sentiment/positive_prompts/')

models = {}
for alpha in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.2, 3.5, 4.0, 5.0, 10.0, 20.0, 30.0]:
    models[f'Layer_24_alpha_{alpha}'] = {
        'pos_path': POS_DIR / f'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_{alpha}/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
    }

pos_prompts_res = read_sentiment_results({m: p['pos_path'] for m,p in models.items()})
negative_steering_res = {}
#assert set(neutral_prompts_res.keys()) == set(pos_prompts_res.keys())
for model in pos_prompts_res.keys():
    negative_steering_res[model] = {
        #'neutral_prompts': neutral_prompts_res[model]['positive_proportion']*100,
        'pos_prompts': pos_prompts_res[model]['positive_proportion']*100,
#         'dist-1': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-1'),
#         'dist-2': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-2'),
#         'dist-3': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-3'),
#         'perplexity': weighted_average(neutral_prompts_res, pos_prompts_res, 'perplexity'),
    }
pd.DataFrame(negative_steering_res).transpose().sort_values(by='pos_prompts', ascending=False).round(2)

100%|██████████| 13/13 [00:02<00:00,  5.62it/s]


,pos_prompts
Layer_24_alpha_0.5,94.64
Layer_24_alpha_1.0,93.97
Layer_24_alpha_1.5,93.32
Layer_24_alpha_2.0,92.59
Layer_24_alpha_2.5,91.89
Layer_24_alpha_3.0,91.09
Layer_24_alpha_3.2,90.71
Layer_24_alpha_3.5,90.25
Layer_24_alpha_4.0,89.40
Layer_24_alpha_5.0,87.35


In [4]:
from typing import Dict
def return_perplexity_of_sentiment_model(models_dict: Dict[str, str]):
    res = {}
    for model in tqdm(models_dict):
        path_to_model = models_dict[model]
        if path_to_model == 'None':
            res[model] = {'ppl': -1}
        else:
            with open(os.path.join(path_to_model, 'eval_results_lm.txt')) as f:
                for line in f:
                    res[model] = {'ppl': float(line.split(' = ')[1])}
                    break
    return res
    

In [35]:
## Different Language Model experiments

# results corresponding to the bottom  half of Table 3

POS_DIR = Path('../generations/sentiment/positive_prompts/')
model_dir = '../models/experts/sentiment/'
source_sentiment = 'positive'
target_sentiment = 'negative'

models = {
    'DExperts (anti-only)': {
        #'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_anti_only/prompted_gens_dexperts.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_anti_only/prompted_gens_dexperts.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}",
        'expert_ppl_path': 'None',
    },
    'DExperts (large)': {
        #'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts/prompted_gens_dexperts.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts/prompted_gens_dexperts.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}",
        'expert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{target_sentiment}",
    },
    'DExperts small experts (anti-only)': {
        #'neutral_path': NEUTRAL_DIR / 'small_experts/negative/dexperts_anti_only/prompted_gens_dexperts.jsonl',
        'pos_path': POS_DIR / 'small_experts/negative/dexperts_anti_only/prompted_gens_dexperts.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/small/finetuned_gpt2_{source_sentiment}",
        'expert_ppl_path': 'None',
    },
    'DExperts small experts': {
        #'neutral_path': NEUTRAL_DIR / 'small_experts/negative/dexperts/prompted_gens_dexperts.jsonl',
        'pos_path': POS_DIR / 'small_experts/negative/dexperts/prompted_gens_dexperts.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/small/finetuned_gpt2_{source_sentiment}",
        'expert_ppl_path': f"{model_dir}/small/finetuned_gpt2_{target_sentiment}",
    },
    'Layer 24 (anti-only) 3 epochs': {
        #'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24",
        'expert_ppl_path': 'None',
    },
    'Layer 24 3 epochs': {
        #'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24",
        'expert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{target_sentiment}_experimental_freeze_emb_and_lmhead_layers24",
    },
    'Layer 24 (anti-only) 50 epochs': {
        #'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_epochs50/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24_50",
        'expert_ppl_path': 'None',
    },
    'Layer 24 50 epochs': {
        #'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert_epochs50/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24_50",
        'expert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{target_sentiment}_experimental_freeze_emb_and_lmhead_layers24_50",
    },
    'Layer 24 (anti-only) 100 epochs': {
        #'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_epochs100/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24_100",
        'expert_ppl_path': 'None',
    },
    'Layer 24 100 epochs': {
        #'neutral_path': NEUTRAL_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'pos_path': POS_DIR / 'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_24_freeze_emb_lm_head_with_expert_epochs100/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'antiexpert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24_100",
        'expert_ppl_path': f"{model_dir}/large/finetuned_gpt2_{target_sentiment}_experimental_freeze_emb_and_lmhead_layers24_100",
    }
}

# read sentiment control results
pos_prompts_res = read_sentiment_results({m: p['pos_path'] for m,p in models.items()})
antiexpert_ppl_res = return_perplexity_of_sentiment_model({m: p['antiexpert_ppl_path'] for m,p in models.items()})
expert_ppl_res = return_perplexity_of_sentiment_model({m: p['expert_ppl_path'] for m,p in models.items()})

negative_steering_res = {}
for model in pos_prompts_res.keys():
    negative_steering_res[model] = {
        'pos_prompts': pos_prompts_res[model]['positive_proportion']*100,
        'antiexpert_ppl': antiexpert_ppl_res[model]['ppl'],
        'expert_ppl': expert_ppl_res[model]['ppl'],
    }
    
pd.DataFrame(negative_steering_res).transpose().sort_values(by='pos_prompts', ascending=False).round(2)



100%|██████████| 10/10 [00:00<00:00, 24892.01it/s]


,pos_prompts,antiexpert_ppl,expert_ppl
DExperts (anti-only),92.29,18.81,-1.00
Layer 24 (anti-only) 3 epochs,91.05,42.09,-1.00
DExperts small experts (anti-only),90.93,40.04,-1.00
Layer 24 3 epochs,90.71,42.09,43.80
Layer 24 100 epochs,68.21,7.69,7.20
Layer 24 50 epochs,64.32,18.01,17.93
Layer 24 (anti-only) 100 epochs,56.00,7.69,-1.00
DExperts small experts,46.55,40.04,41.09
Layer 24 (anti-only) 50 epochs,40.44,18.01,-1.00
DExperts (large),33.95,18.81,18.58


{'ppl': 7.69039240771082}

In [ ]:
#pos -> neg
model_dir = '../models/experts/sentiment/'
source_sentiment = 'positive'
target_sentiment = 'negative'
model_path_to_expert_ppls = {
    'DExperts (anti-only)': 'None'#f"{model_dir}/large/finetuned_gpt2_{target_sentiment}",
    'Layer 24 (anti-only) 3 epochs': 'None'#f"{model_dir}/large/finetuned_gpt2_{target_sentiment}_experimental_freeze_emb_and_lmhead_layers24",
    'DExperts small experts (anti-only)': 'None'#f"{model_dir}/small/finetuned_gpt2_{target_sentiment}",
    'Layer 24 3 epochs': f"{model_dir}/large/finetuned_gpt2_{target_sentiment}_experimental_freeze_emb_and_lmhead_layers24",
    'Layer 24 100 epochs': f"{model_dir}/large/finetuned_gpt2_{target_sentiment}_experimental_freeze_emb_and_lmhead_layers24_50",
    'Layer 24 50 epochs': f"{model_dir}/large/finetuned_gpt2_{target_sentiment}_experimental_freeze_emb_and_lmhead_layers24_100",
    'Layer 24 (anti-only) 100 epochs': 'None'#f"{model_dir}/large/finetuned_gpt2_{target_sentiment}_experimental_freeze_emb_and_lmhead_layers24_100",
    'DExperts small experts': f"{model_dir}/small/finetuned_gpt2_{target_sentiment}",
    'Layer 24 (anti-only) 50 epochs': 'None',#f"{model_dir}/large/finetuned_gpt2_{target_sentiment}_experimental_freeze_emb_and_lmhead_layers24_50",
    'DExperts (large)': f"{model_dir}/large/finetuned_gpt2_{target_sentiment}",
}

model_path_to_antiexpert_ppls = {
    'DExperts (anti-only)': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}",
    'Layer 24 (anti-only) 3 epochs': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24",
    'DExperts small experts (anti-only)': f"{model_dir}/small/finetuned_gpt2_{source_sentiment}",
    'Layer 24 3 epochs': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24",
    'Layer 24 100 epochs': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24_50",
    'Layer 24 50 epochs': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24_100",
    'Layer 24 (anti-only) 100 epochs': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24_100",
    'DExperts small experts': f"{model_dir}/small/finetuned_gpt2_{source_sentiment}",
    'Layer 24 (anti-only) 50 epochs': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}_experimental_freeze_emb_and_lmhead_layers24_50",
    'DExperts (large)': f"{model_dir}/large/finetuned_gpt2_{source_sentiment}",
}

pos_prompts_expert_ppl_res = return_perplexity_of_sentiment_model({m: p['pos_path'] for m,p in models.items()})

negative_steering_res = {}
for model in pos_prompts_res.keys():
    negative_steering_res[model] = {
        'pos_prompts': pos_prompts_res[model]['positive_proportion']*100,
    }
    
pd.DataFrame(negative_steering_res).transpose().sort_values(by='pos_prompts', ascending=False).round(2)



In [5]:
from typing import Dict
def return_perplexity_of_sentiment_model(models_dict: Dict[str, str]):
    res = {}
    for model in tqdm(models_dict):
        path_to_model = models_dict[model]
        if path_to_model == 'None':
            res[model] = {'ppl': -1}
        else:
            with open(os.path.join(path_to_model, 'eval_results_lm.txt')) as f:
                for line in f:
                    res[model] = {'ppl': float(line.split(' = ')[1])}
                    break
    return res


POS_DIR = Path('../generations/sentiment/positive_prompts/')
model_dir = '../models/experts/sentiment/'
source_sentiment = 'positive'
target_sentiment = 'negative'

models = {
    'GPT2 large': {
        'positive_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_positive_100_maxepochs',
        'negative_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_negative_100_maxepochs',
    },
    'GPT2 small': {
        'positive_path': f'{model_dir}/small/finetuned_gpt2_train_val_sst_positive_100_maxepochs',
        'negative_path': f'{model_dir}/small/finetuned_gpt2_train_val_sst_negative_100_maxepochs',
    }
}

for i in range(0, 36):
    models[f'layer{i}'] = {
        'positive_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_positive_experimental_freeze_emb_and_lmhead_layers{i}_100_maxepochs',
        'negative_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_negative_experimental_freeze_emb_and_lmhead_layers{i}_100_maxepochs',
    }

positive_ppl_res = return_perplexity_of_sentiment_model({m: p['positive_path'] for m,p in models.items()})
negative_ppl_res = return_perplexity_of_sentiment_model({m: p['negative_path'] for m,p in models.items()})

negative_steering_res = {}
for model in positive_ppl_res.keys():
    negative_steering_res[model] = {
        'positive_ppl': positive_ppl_res[model]['ppl'],
        'negative_ppl': negative_ppl_res[model]['ppl'],
    }
    
pd.DataFrame(negative_steering_res).transpose().round(2)



100%|██████████| 38/38 [00:00<00:00, 32362.14it/s]


,positive_ppl,negative_ppl
GPT2 large,31.35,30.51
GPT2 small,42.89,43.15
layer0,34.50,33.45
layer1,33.73,32.80
layer2,33.62,32.81
layer3,33.65,32.94
layer4,34.10,33.36
layer5,33.85,33.04
layer6,33.72,32.96
layer7,33.66,33.09


In [6]:
### New language models train_val_sst positive
# results corresponding to the bottom  half of Table 3

model_dir = '../models/experts/sentiment/'
POS_DIR = Path('../generations/sentiment/positive_prompts/')

models = {
    "DExperts Small" : {
        'pos_path': POS_DIR / f"small_experts/negative/dexperts/train_val_sst/prompted_gens_dexperts.jsonl",
        'positive_ppl_path': f'{model_dir}/small/finetuned_gpt2_train_val_sst_positive_100_maxepochs',
        'negative_ppl_path': f'{model_dir}/small/finetuned_gpt2_train_val_sst_negative_100_maxepochs',
    },
    "DExperts Small (anti-only)" : {
        'pos_path': POS_DIR / f"small_experts/negative/dexperts_anti_only/train_val_sst/prompted_gens_dexperts.jsonl",
        'positive_ppl_path': f'{model_dir}/small/finetuned_gpt2_train_val_sst_positive_100_maxepochs',
        'negative_ppl_path': 'None',
    },
    "DExperts Small (expert-only)" : {
        'pos_path': POS_DIR / f"small_experts/negative/dexperts_expert_only/train_val_sst/prompted_gens_dexperts.jsonl",
        'positive_ppl_path': 'None',
        'negative_ppl_path': f'{model_dir}/small/finetuned_gpt2_train_val_sst_negative_100_maxepochs',
    },
    "DExperts" : {
        'pos_path': POS_DIR / f"large_experts/negative/dexperts/train_val_sst/prompted_gens_dexperts.jsonl",
        'positive_ppl_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_positive_100_maxepochs',
        'negative_ppl_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_negative_100_maxepochs',
    },
    "DExperts (anti-only)" : {
        'pos_path': POS_DIR / f"large_experts/negative/dexperts_anti_only/train_val_sst/prompted_gens_dexperts.jsonl",
        'positive_ppl_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_positive_100_maxepochs',
        'negative_ppl_path': 'None',
    },
    "DExperts (expert-only)" : {
        'pos_path': POS_DIR / f"large_experts/negative/dexperts_expert_only/train_val_sst/prompted_gens_dexperts.jsonl",
        'positive_ppl_path': 'None', 
        'negative_ppl_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_negative_100_maxepochs',
    },
    
}

for layer_num in [2, 12, 22, 32]:
    models[f'Layer_{layer_num}'] = {
        'pos_path': POS_DIR / f'large_experts/negative/dexperts_steer/steering_large_gpt2/alpha_3.2/layer_{layer_num}_freeze_emb_lm_head_with_expert_epochs100_train_val_sst/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'positive_ppl_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_positive_experimental_freeze_emb_and_lmhead_layers{layer_num}_100_maxepochs',
        'negative_ppl_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_negative_experimental_freeze_emb_and_lmhead_layers{layer_num}_100_maxepochs',
    }
    
    models[f'Layer_{layer_num} (anti-only)'] = {
        'pos_path': POS_DIR / f'large_experts/negative/dexperts_steer_anti_only/steering_large_gpt2/alpha_3.2/layer_{layer_num}_freeze_emb_lm_head_epochs100_train_val_sst/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'positive_ppl_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_positive_experimental_freeze_emb_and_lmhead_layers{layer_num}_100_maxepochs',
        'negative_ppl_path': 'None',
    }
    
    models[f'Layer_{layer_num} (expert-only)'] = {
        'pos_path': POS_DIR / f'large_experts/negative/dexperts_steer_expert_only/steering_large_gpt2/alpha_3.2/layer_{layer_num}_freeze_emb_lm_head_epochs100_train_val_sst/combine_at_layer/prompted_gens_dexperts-steer.jsonl',
        'positive_ppl_path': 'None',
        'negative_ppl_path': f'{model_dir}/large/finetuned_gpt2_train_val_sst_negative_experimental_freeze_emb_and_lmhead_layers{layer_num}_100_maxepochs',
    }

pos_prompts_res = read_sentiment_results({m: p['pos_path'] for m,p in models.items()})
antiexpert_ppl_res = return_perplexity_of_sentiment_model({m: p['positive_ppl_path'] for m,p in models.items()})
expert_ppl_res = return_perplexity_of_sentiment_model({m: p['negative_ppl_path'] for m,p in models.items()})
negative_steering_res = {}
#assert set(neutral_prompts_res.keys()) == set(pos_prompts_res.keys())
for model in pos_prompts_res.keys():
    negative_steering_res[model] = {
        #'neutral_prompts': neutral_prompts_res[model]['positive_proportion']*100,
        'pos_prompts': pos_prompts_res[model]['positive_proportion']*100,
        'antiexpert_ppl': antiexpert_ppl_res[model]['ppl'],
        'expert_ppl': expert_ppl_res[model]['ppl'],
#         'dist-1': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-1'),
#         'dist-2': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-2'),
#         'dist-3': weighted_average(neutral_prompts_res, pos_prompts_res, 'dist-3'),
#         'perplexity': weighted_average(neutral_prompts_res, pos_prompts_res, 'perplexity'),
    }
pd.DataFrame(negative_steering_res).transpose().sort_values(by='pos_prompts', ascending=False).round(2)

100%|██████████| 18/18 [00:00<00:00, 23952.24it/s]


,pos_prompts,antiexpert_ppl,expert_ppl
DExperts Small (anti-only),90.92,42.89,-1.00
DExperts (anti-only),90.66,31.35,-1.00
Layer_22 (anti-only),86.86,34.61,-1.00
Layer_32 (anti-only),83.44,36.71,-1.00
Layer_32,78.99,36.71,36.98
Layer_12 (anti-only),77.67,34.23,-1.00
Layer_2,69.26,33.62,32.81
Layer_2 (anti-only),68.68,33.62,-1.00
Layer_12,67.75,34.23,33.93
Layer_32 (expert-only),63.69,-1.00,36.98
